<a href="https://colab.research.google.com/github/koksal100/Deep-Learning/blob/main/Multiclass_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [124]:
import pandas as pd
import tensorflow as tf

In [125]:
from google.colab import drive
drive.mount('/content/drive')
df=pd.read_csv('/content/drive/My Drive/Colab Notebooks/FM 2023.csv', encoding='ISO-8859-1')



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [126]:
df

,ï»¿Name,Position,Age,ca,pa,Nationality,Club,Corners,Crossing,Dribbling,...,World reputation,Race,RCA,Colour of skin,Date of birth,Number of national team appearances,Goals scored for the national team,Salary,Rental club,UID
0,Kevin De Bruyne,M/AM RLC,31,189,189,Belgium,Manchester City,14,19,15,...,9400,Northern_European,181,4,1991/6/28,91,24,394372.0,NaN,18004457
1,Kylian MbappÃ©,AM/S RL,23,188,197,France,Paris Saint-Germain,13,13,18,...,9248,African_Caribbean,172,13,1998/12/20,57,27,1035616.0,NaN,85139014
2,Robert Lewandowski,S,33,186,190,Poland,Barcelona,3,8,13,...,9250,Northern_European,183,3,1988/8/21,132,76,345204.0,NaN,719601
3,Erling Haaland,S,22,185,195,"Norway,England",Manchester City,7,10,14,...,8750,Northern_European,185,2,2000/7/21,21,20,394372.0,NaN,29179241
4,Mohamed Salah,AM/S RL,30,185,187,Egypt,Liverpool,12,14,17,...,8750,North_African__Middle_Eastern,181,9,1992/6/15,85,47,405971.0,NaN,98028755
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8447,Joe Ashton,D L,16,45,135,England,Burnley,3,4,5,...,236,Northern_European,41,3,2006/6/26,0,0,203.0,NaN,2000210572
8448,River Ries,S,17,45,135,Germany,Karlsruher SC,2,2,11,...,462,Northern_European,33,255,2005/4/3,0,0,402.0,NaN,2000197059
8449,Halilcan DoÄan,D C,23,45,-75,Turkey,Osmaniyespor Futbol KulÃ¼bÃ¼,1,1,1,...,1145,Northern_European,41,3,1999/2/11,0,0,82.0,NaN,2000047739
8450,Adijat Sefer,S,17,45,135,Germany,TSG Hoffenheim,3,4,12,...,474,Northern_European,40,255,2005/1/20,0,0,646.0,NaN,2000197383


In [127]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8452 entries, 0 to 8451
Data columns (total 98 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   ï»¿Name                              8452 non-null   object 
 1   Position                             8452 non-null   object 
 2   Age                                  8452 non-null   int64  
 3   ca                                   8452 non-null   int64  
 4   pa                                   8452 non-null   int64  
 5   Nationality                          8452 non-null   object 
 6   Club                                 8345 non-null   object 
 7   Corners                              8452 non-null   int64  
 8   Crossing                             8452 non-null   int64  
 9   Dribbling                            8452 non-null   int64  
 10  Finishing                            8452 non-null   int64  
 11  First Touch                   

In [128]:
df.isnull().sum().sort_values(ascending=False)

Rental club      7457
Salary            107
Club              107
ï»¿Name             0
Loyal               0
                 ... 
Off The Ball        0
Leadership          0
Flair               0
Determination       0
UID                 0
Length: 98, dtype: int64

In [129]:
#Rental Club and Club informations are already in the type of string, so they do no mean any numeric information.
#Let's fill the na value of salaries with the mean of it.

In [130]:
df["Salary"].fillna(df["Salary"].mean(),inplace=True)

In [131]:
df["Salary"].isnull().sum()

0

In [132]:
#Let's look at the string column names
stringColumns=df.select_dtypes(include="object").columns
stringColumns

Index(['ï»¿Name', 'Position', 'Nationality', 'Club', 'Race', 'Date of birth',
       'Rental club'],
      dtype='object')

In [133]:
#Firstly let's group positions into 4 seperate as mentioned before.
#before that detect the best position of each player

In [134]:
positions_columns=["GK","DL","DC","DR","WBL","WBR","DM","ML","MC","MR","AML","AMC","AMR","ST"]

In [135]:
df[positions_columns].transpose()

,0,1,2,3,4,5,6,7,8,9,...,8442,8443,8444,8445,8446,8447,8448,8449,8450,8451
GK,1,1,1,1,1,1,1,20,1,1,...,1,1,1,1,1,1,1,1,1,1
DL,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,20,1,1,1,1
DC,1,1,1,1,1,1,1,1,20,1,...,1,1,20,1,1,1,1,20,1,1
DR,1,1,1,1,1,1,1,1,1,1,...,1,1,20,1,20,1,1,1,1,1
WBL,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
WBR,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,15,1,1,1,1,1
DM,12,1,1,1,1,1,1,1,12,1,...,20,1,1,1,1,1,1,1,1,1
ML,15,10,1,1,1,1,1,1,1,8,...,1,15,1,20,1,1,1,1,1,1
MC,20,1,1,1,1,1,11,1,1,12,...,1,1,1,1,1,1,1,1,1,1
MR,15,10,1,1,1,1,1,1,1,8,...,1,14,1,1,1,1,1,1,1,1


In [136]:
best_positions=df[positions_columns].idxmax(axis=1)
best_positions

0        MC
1        ST
2        ST
3        ST
4       AMR
       ... 
8447     DL
8448     ST
8449     DC
8450     ST
8451     ST
Length: 8452, dtype: object

In [137]:
df["position"]=best_positions

In [138]:
#Now we can convert them into for seperate groups

In [139]:
df["position"].value_counts()

DC     1438
MC     1220
ST     1201
AMC     754
GK      697
AML     686
DM      656
AMR     600
DR      487
DL      474
MR       66
ML       63
WBL      56
WBR      54
Name: position, dtype: int64

In [140]:
def converter(x):
    if x[0:2] == "DM" or x[0]=="M" or x[0]=="A":
      return "Middlefielder"
    elif x[0]=="D" or x[0]=="W":
      return "Defender"
    elif x[0]=="S":
      return "Forward"
    elif x[0]=="G":
      return "Goalkeeper"

In [141]:
df["position group"]=df["position"].apply(converter)

In [142]:
df["position group"]

0       Middlefielder
1             Forward
2             Forward
3             Forward
4       Middlefielder
            ...      
8447         Defender
8448          Forward
8449         Defender
8450          Forward
8451          Forward
Name: position group, Length: 8452, dtype: object

In [143]:
pd.get_dummies(df["position group"])

,Defender,Forward,Goalkeeper,Middlefielder
0,0,0,0,1
1,0,1,0,0
2,0,1,0,0
3,0,1,0,0
4,0,0,0,1
...,...,...,...,...
8447,1,0,0,0
8448,0,1,0,0
8449,1,0,0,0
8450,0,1,0,0


In [144]:
df=pd.concat([df,pd.get_dummies(df["position group"])],axis=1)

In [145]:
stirngColumns=df.select_dtypes(include="object").columns

In [146]:
stirngColumns

Index(['ï»¿Name', 'Position', 'Nationality', 'Club', 'Race', 'Date of birth',
       'Rental club', 'position', 'position group'],
      dtype='object')

In [147]:
#Now drop all the string columns.
df.drop(stringColumns,axis=1,inplace=True)

In [148]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8452 entries, 0 to 8451
Data columns (total 97 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   Age                                  8452 non-null   int64  
 1   ca                                   8452 non-null   int64  
 2   pa                                   8452 non-null   int64  
 3   Corners                              8452 non-null   int64  
 4   Crossing                             8452 non-null   int64  
 5   Dribbling                            8452 non-null   int64  
 6   Finishing                            8452 non-null   int64  
 7   First Touch                          8452 non-null   int64  
 8   Free Kick Taking                     8452 non-null   int64  
 9   Heading                              8452 non-null   int64  
 10  Long Shots                           8452 non-null   int64  
 11  Long Throws                   

In [149]:
df.drop(["position","position group"],axis=1,inplace=True)

In [150]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8452 entries, 0 to 8451
Data columns (total 95 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   Age                                  8452 non-null   int64  
 1   ca                                   8452 non-null   int64  
 2   pa                                   8452 non-null   int64  
 3   Corners                              8452 non-null   int64  
 4   Crossing                             8452 non-null   int64  
 5   Dribbling                            8452 non-null   int64  
 6   Finishing                            8452 non-null   int64  
 7   First Touch                          8452 non-null   int64  
 8   Free Kick Taking                     8452 non-null   int64  
 9   Heading                              8452 non-null   int64  
 10  Long Shots                           8452 non-null   int64  
 11  Long Throws                   

In [151]:
y=df[["Defender","Forward","Goalkeeper","Middlefielder"]]

In [152]:
X=df.drop(y.columns,axis=1)

In [153]:
from sklearn.model_selection import train_test_split

In [154]:
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.33, random_state=42)

In [155]:
from sklearn.preprocessing import StandardScaler

In [156]:
scaler=StandardScaler()

In [157]:
x_train_norm=scaler.fit_transform(X_train)
x_test_norm=scaler.transform(X_test)

In [158]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(128, activation='relu', input_shape=(91,)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(4, activation='softmax')  # 4 sınıfı temsil eder
])

# Modeli derleme ve özeti gösterme
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(x_train_norm,y_train,epochs=50,verbose=1)

Epoch 1/50
177/177 [==============================] - 1s 2ms/step - loss: 0.2651 - accuracy: 0.8976
Epoch 2/50
177/177 [==============================] - 0s 2ms/step - loss: 0.1058 - accuracy: 0.9583
Epoch 3/50
177/177 [==============================] - 0s 2ms/step - loss: 0.0740 - accuracy: 0.9702
Epoch 4/50
177/177 [==============================] - 1s 3ms/step - loss: 0.0561 - accuracy: 0.9772
Epoch 5/50
177/177 [==============================] - 0s 3ms/step - loss: 0.0412 - accuracy: 0.9845
Epoch 6/50
177/177 [==============================] - 0s 3ms/step - loss: 0.0287 - accuracy: 0.9915
Epoch 7/50
177/177 [==============================] - 0s 3ms/step - loss: 0.0211 - accuracy: 0.9949
Epoch 8/50
177/177 [==============================] - 0s 3ms/step - loss: 0.0143 - accuracy: 0.9975
Epoch 9/50
177/177 [==============================] - 0s 2ms/step - loss: 0.0086 - accuracy: 0.9989
Epoch 10/50
177/177 [==============================] - 0s 3ms/step - loss: 0.0079 - accuracy: 0.9981

In [159]:
model.evaluate(x_test_norm,y_test)

88/88 [==============================] - 0s 3ms/step - loss: 0.1907 - accuracy: 0.9670


[0.1906987726688385, 0.967025101184845]